# National Factors and Their Effect on U.S. Home Prices: A Study

**Task -** Retrieve publicly accessible data for key factors that have an impact on U.S. home prices. Develop a data science model that explains the effect of these factors on home prices, using the S&P Case-Shiller Home Price Index as a benchmark, during the past two decades



Study aims to understand the impact of national factors on U.S. home prices in the past 20 years.
  
Selected variables that impact home prices, including:

    1.Unemployment rate
    2.Median household income
    3.Construction prices
    4.Consumer Price Index
    5.Interest rates
    6.Housing subsidies
    7.Economic Policy Uncertainity

--> Approach is to gather publicly available data for these variables and build a data science model.

--> Model will explain how these factors have influenced home prices during the past 20 years.
    
--> Analysis will provide valuable insights into housing market behavior and inform policy decisions, investment strategies, and other considerations related to housing in the U.S.

--> Using S&P Case-Shiller Home Price Index as a proxy for home prices to ensure robust and widely recognized measure of housing market trends.


--> Most of the data is downloaded from [https://fred.stlouisfed.org/].



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Loading Interest Rates
df_fed_rate = pd.read_csv('FEDFUNDS.csv')
df_fed_rate['DATE'] = pd.to_datetime(df_fed_rate['DATE'])
df_fed_rate = df_fed_rate[df_fed_rate['DATE'] >= '2004-01-01']
df_fed_rate.head()

,DATE,FEDFUNDS
594,2004-01-01,1.00
595,2004-02-01,1.01
596,2004-03-01,1.00
597,2004-04-01,1.00
598,2004-05-01,1.00


In [ ]:
df_fed_rate.shape

(245, 2)

In [ ]:
# Date Filter Function

def filter_by_date(df, column_name, start_date, end_date=None):
    df[column_name] = pd.to_datetime(df[column_name]) # convert the date column to datetime format
    if end_date is None:
        return df[df[column_name] >= start_date] # filter the dataset to include only rows with a date on or after start_date
    else:
        return df[(df[column_name] >= start_date) & (df[column_name] <= end_date)] # filter the dataset to include only rows between start_date and end_date


In [ ]:
# Loading Construction Materials
df_cons_price_index = pd.read_csv("/content/WPUSI012011.csv")
df_cons_price_index = df_cons_price_index.rename(columns={"WPUSI012011": "construction_materials"})
df_cons_price_index = filter_by_date(df_cons_price_index, 'DATE', '2004-01-01')
df_cons_price_index.head()

,DATE,construction_materials
684,2004-01-01,150.0
685,2004-02-01,153.4
686,2004-03-01,156.5
687,2004-04-01,160.1
688,2004-05-01,162.7


In [ ]:
df_cons_price_index.shape

(244, 2)

In [ ]:
# Loading Income
df_CPI = pd.read_csv("/content/DSPIC96.csv")
df_CPI = df_CPI.rename(columns={"DSPIC96": "Income"})
df_CPI = filter_by_date(df_CPI, 'DATE', '2004-01-01')
df_CPI.head()


,DATE,Income
540,2004-01-01,11051.2
541,2004-02-01,11071.0
542,2004-03-01,11115.6
543,2004-04-01,11153.3
544,2004-05-01,11208.9


In [ ]:
df_CPI.shape

(244, 2)

In [ ]:
# Loading Total Units
df_units = pd.read_csv("/content/COMPUTSA.csv")
df_units= df_units.rename(columns={"COMPUTSA": "Total Units"})
df_units = filter_by_date(df_units, 'DATE', '2004-01-01')
df_units.head()

,DATE,Total Units
432,2004-01-01,1709.0
433,2004-02-01,1718.0
434,2004-03-01,1794.0
435,2004-04-01,1938.0
436,2004-05-01,1893.0


In [ ]:
df_units.shape

(244, 2)

In [ ]:
# Loading Housing Subsidies
df_subsidies = pd.read_csv("/content/L312051A027NBEA.csv")
df_subsidies= df_subsidies.rename(columns={"L312051A027NBEA": "Subsidy"})
df_subsidies = filter_by_date(df_subsidies, 'DATE', '2004-01-01')
df_subsidies.head()

,DATE,Subsidy
44,2004-01-01,27.201
45,2005-01-01,27.651
46,2006-01-01,28.604
47,2007-01-01,29.512
48,2008-01-01,29.876


In [ ]:
df_subsidies.shape

(19, 2)

In [ ]:
# Loading Unemployment Rate
df_unemployment = pd.read_csv("/content/LNS14000024.csv")
df_unemployment= df_unemployment.rename(columns={"LNS14000024": "Unemployment"})
df_unemployment = filter_by_date(df_unemployment, 'DATE', '2004-01-01')
df_unemployment.head()

,DATE,Unemployment
672,2004-01-01,5.1
673,2004-02-01,5.0
674,2004-03-01,5.2
675,2004-04-01,5.0
676,2004-05-01,5.0


In [ ]:
df_unemployment.shape

(244, 2)

In [ ]:
# Loading Inflation (CPI)
df_inflation = pd.read_csv("/content/CORESTICKM159SFRBATL.csv")
df_inflation= df_inflation.rename(columns={"CORESTICKM159SFRBATL": "Inflation"})
df_inflation = filter_by_date(df_inflation, 'DATE', '2004-01-01')
df_inflation.head()

,DATE,Inflation
432,2004-01-01,1.949813
433,2004-02-01,2.037157
434,2004-03-01,2.126567
435,2004-04-01,2.247883
436,2004-05-01,2.228612


In [ ]:
df_inflation.shape

(244, 2)

In [ ]:
# Loading EPU (CPI)
df_epu = pd.read_csv("/content/USEPUINDXD(1).csv")
df_epu= df_epu.rename(columns={"USEPUINDXD": "EPU"})
df_epu['DATE'] = pd.to_datetime(df_epu['DATE'])
df_epu = df_epu.set_index('DATE').resample('M').mean().reset_index()
df_epu.head()

,DATE,EPU
0,2004-01-31,93.051290
1,2004-02-29,83.246552
2,2004-03-31,63.193226
3,2004-04-30,59.875000
4,2004-05-31,76.701613


In [ ]:
# Loading Median Household
df_household = pd.read_csv("/content/MEHOINUSA672N.csv")
df_household= df_household.rename(columns={"MEHOINUSA672N": "Household_income"})
df_household.head()

,DATE,Household_income
0,2004-01-01,65760
1,2005-01-01,66780
2,2006-01-01,67520
3,2007-01-01,68610
4,2008-01-01,66280


In [ ]:
df_household.shape

(19, 2)

**On analysis of the impact of these columns, I've picked up the columns that have the most impact to the housing prices and left those columns which I don't think have significant impact.**

In [ ]:
# Merging Dataframes
df = pd.DataFrame()
df_bymonth = [df_fed_rate,df_cons_price_index,df_CPI,df_units,df_inflation,df_unemployment]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()

(245, 6)


,FEDFUNDS,construction_materials,Income,Total Units,Inflation,Unemployment
DATE,,,,,,
2004-01-01,1.00,150.0,11051.2,1709.0,1.949813,5.1
2004-02-01,1.01,153.4,11071.0,1718.0,2.037157,5.0
2004-03-01,1.00,156.5,11115.6,1794.0,2.126567,5.2
2004-04-01,1.00,160.1,11153.3,1938.0,2.247883,5.0
2004-05-01,1.00,162.7,11208.9,1893.0,2.228612,5.0


In [ ]:
# Checking null values
df.isna().sum()

FEDFUNDS                  0
construction_materials    1
Income                    1
Total Units               1
Inflation                 1
Unemployment              1
dtype: int64

In [ ]:
# Dropping them.
df.dropna(inplace = True)

In [ ]:
df.index = pd.to_datetime(df.index)
df['Year'] = df.index.year
df['Month'] = df.index.month
df.head()

,FEDFUNDS,construction_materials,Income,Total Units,Inflation,Unemployment,Year,Month
DATE,,,,,,,,
2004-01-01,1.00,150.0,11051.2,1709.0,1.949813,5.1,2004,1
2004-02-01,1.01,153.4,11071.0,1718.0,2.037157,5.0,2004,2
2004-03-01,1.00,156.5,11115.6,1794.0,2.126567,5.2,2004,3
2004-04-01,1.00,160.1,11153.3,1938.0,2.247883,5.0,2004,4
2004-05-01,1.00,162.7,11208.9,1893.0,2.228612,5.0,2004,5


In [ ]:
# checking if all of the null values have been replaced/dropped
df.isna().sum()

FEDFUNDS                  0
construction_materials    0
Income                    0
Total Units               0
Inflation                 0
Unemployment              0
Year                      0
Month                     0
dtype: int64

In [ ]:
df_epu['Year'] = df_epu['DATE'].dt.year
df_epu['Month'] = df_epu['DATE'].dt.month
df_epu.drop(columns= ['DATE'], inplace = True)
df_epu.head()

,EPU,Year,Month
0,93.051290,2004,1
1,83.246552,2004,2
2,63.193226,2004,3
3,59.875000,2004,4
4,76.701613,2004,5


In [ ]:
 # Merging the relevant dataframes
 df_final = pd.merge(df, df_epu, on=['Year','Month'], how='inner')

In [ ]:
# final dataframe
df_final.head()

,FEDFUNDS,construction_materials,Income,Total Units,Inflation,Unemployment,Year,Month,EPU
0,1.00,150.0,11051.2,1709.0,1.949813,5.1,2004,1,93.051290
1,1.01,153.4,11071.0,1718.0,2.037157,5.0,2004,2,83.246552
2,1.00,156.5,11115.6,1794.0,2.126567,5.2,2004,3,63.193226
3,1.00,160.1,11153.3,1938.0,2.247883,5.0,2004,4,59.875000
4,1.00,162.7,11208.9,1893.0,2.228612,5.0,2004,5,76.701613


In [ ]:
# Downloading the dataframe.
from google.colab import files
df.to_csv("cleaned_data.csv", encoding='utf-8', index=False)
files.download('cleaned_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Memory usage of dataframes
df_final.memory_usage()

Index                      128
FEDFUNDS                  1952
construction_materials    1952
Income                    1952
Total Units               1952
Inflation                 1952
Unemployment              1952
Year                       976
Month                      976
EPU                       1952
dtype: int64